In [1]:
# Creates relations of BusinessArea <> ConceptualEntity
import sqlite3
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sys
sys.path.append('../../')
from user_packages import hashing

In [2]:
# Set script variables
source_file_name = 'Org_Dictionary(BusinessArea_Entity_relations).csv'
source_file_path = '../../imports/concept'

record_source = 'BusinessAreaConceptualEntityRelationsCSV'

In [3]:
conn = sqlite3.connect('../../full_metadata.db')
cur = conn.cursor()

In [12]:
# read table to df
df = pd.read_csv(os.path.join(source_file_path, source_file_name))
df = df.replace({np.nan: None})

#df

In [13]:
# Create additional fields
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()

df['ConceptualEntityKeyPhrase'] = df.apply(lambda row: row['EntityName'], axis=1)
df['BusinessAreaKeyPhrase'] = df.apply(lambda row: row['BusinessName'] + '.' + row['BusinessAreaName'], axis=1)
df['BusinessAreaConceptualEntityKeyPhrase'] = df.apply(lambda row: row['BusinessAreaKeyPhrase'] + ':' + row['ConceptualEntityKeyPhrase'], axis=1)


In [14]:
# Hash fields

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ConceptualEntityHashKey'
  , columns = ['ConceptualEntityKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessAreaHashKey'
  , columns = ['BusinessAreaKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessAreaConceptualEntityHashKey'
  , columns = ['BusinessAreaConceptualEntityKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = [
    'isDeleted'
  ]
)

#df

In [8]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_CSV_BA_to_CERelated")
conn.commit()
df.to_sql('stg_Py_CSV_BA_to_CERelated', conn, if_exists='append', index=False)

59

In [9]:
# Write to the Link (BusinessArea - ConceptualEntity)
sql_query = """
INSERT INTO rv_l_BusinessAreaConceptualEntity
(
  BusinessAreaConceptualEntityHashKey
  , LoadDate
  , RecordSource
  , BusinessAreaHashKey
  , ConceptualEntityHashKey
)
SELECT
  stg.BusinessAreaConceptualEntityHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.BusinessAreaHashKey
  , stg.ConceptualEntityHashKey
FROM
  stg_Py_CSV_BA_to_CERelated stg
WHERE
  stg.BusinessAreaConceptualEntityHashKey IS NOT NULL
  AND BusinessAreaConceptualEntityHashKey NOT IN (
    SELECT BusinessAreaConceptualEntityHashKey
    FROM rv_l_BusinessAreaConceptualEntity
  )
""";
conn.execute(sql_query)
conn.commit()

In [11]:
# Write to the Satellite (rv_s_ConceptualEntityConceptualEntity_Related)
sql_query = """
INSERT INTO rv_s_BusinessAreaConceptualEntity
(
    BusinessAreaConceptualEntityHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , isDeleted
)
SELECT DISTINCT
    stg.BusinessAreaConceptualEntityHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.isDeleted
FROM
  stg_Py_CSV_BA_to_CERelated AS stg
  LEFT OUTER JOIN rv_s_BusinessAreaConceptualEntity AS sat ON (
    stg.BusinessAreaConceptualEntityHashKey = sat.BusinessAreaConceptualEntityHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_BusinessAreaConceptualEntity AS z
      WHERE z.BusinessAreaConceptualEntityHashKey = sat.BusinessAreaConceptualEntityHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()